In [40]:
# Make sure libraries are reloaded when this cell is run
%load_ext autoreload
%autoreload 2

from astropy import units as u

from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import OrbitPlotter3D
import copy
import CustomManeuvres

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
# Define the two coplanar debris
a = 3000+6371 << u.km
ecc = 0 << u.one
inc = 0 << u.deg
raan = 0 << u.deg
argp = 0 << u.deg
nu = 120 << u.deg

deb_1 = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

a = 8000+6371 << u.km
nu = 10 << u.deg

deb_2 = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu)

# Init the otv at debris 1
otv = copy.copy(deb_1)

In [52]:
print(deb_1.epoch.iso)
print(deb_2.epoch.iso)
print(otv.epoch.iso)


2000-01-01 12:00:00.000
2000-01-01 12:00:00.000
2000-01-01 12:00:00.000


In [53]:
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)

In [54]:
# Calculate a basic homnann
import CustomManeuvres
hoh_phas = CustomManeuvres.hohmann_with_phasing(otv, deb_2)
hoh_phas


Target Delta:  33.39846265144412 deg
current_delta :  250.0 deg
t_1:  11473.200782188966 s
orbit_i.nu:  -142.49356463896865 deg
new delta:  33.39846265144409 deg
rv:  (array([-7433873.362744  , -5705538.34678947,        0.        ]), array([ 3970.87859889, -5173.74642826,     0.        ]))
rot_matrix:  [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
dv_a:  [ 398.16573707 -518.77903308    0.        ]
dv_b:  [-357.58197328  465.90154068    0.        ]


Number of impulses: 2, Total cost: 1.241270 km / s

In [56]:
hoh_phas.impulses

((<Quantity 11473.20078219 s>,
  <Quantity [ 398.16573707, -518.77903308,    0.        ] m / s>),
 (<Quantity 6435.93934258 s>,
  <Quantity [-357.58197328,  465.90154068,    0.        ] m / s>))

In [55]:
# Get the transfer time of the hoh_phas
transfer_time = hoh_phas.get_total_time()

# Propagate all debris to the end of the transfer
deb_1 = deb_1.propagate(transfer_time)
deb_2 = deb_2.propagate(transfer_time)

# Propagate the otv using the calculated thrust sequence
transfer_orbit, otv = otv.apply_maneuver(hoh_phas, intermediate=True)

# Plot the results
frame = OrbitPlotter3D()
frame.plot(deb_1)
frame.plot(deb_2)
frame.plot(otv)
frame.plot(transfer_orbit)



In [48]:
print("OTV ANGLE: ", otv.nu.to(u.deg))
print("DEB2 ANGLE: ", deb_2.nu.to(u.deg))

OTV ANGLE:  37.506435361031464 deg
DEB2 ANGLE:  26.041495059184214 deg
